<a href="https://colab.research.google.com/github/Aaron-lobo/Chatbot-Using-Trax/blob/main/Chatbot_Trax.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade tensorflow-probability==0.8
!pip install trax
!pip install t5==0.8.1
!pip list | grep trax

Requirement already up-to-date: tensorflow-probability==0.8 in /usr/local/lib/python3.7/dist-packages (0.8.0)
Processing /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd/gast-0.2.2-cp37-none-any.whl
  Using cached https://files.pythonhosted.org/packages/24/fb/4f92f8c0f40a0d728b4f3d5ec5ff84353e705d8ff5e3e447620ea98b06bd/cloudpickle-1.1.1-py2.py3-none-any.whl
ERROR: tensorflow 2.4.1 has requirement gast==0.3.3, but you'll have gast 0.2.2 which is incompatible.
ERROR: gym 0.17.3 has requirement cloudpickle<1.7.0,>=1.2.0, but you'll have cloudpickle 1.1.1 which is incompatible.
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: cloudpickle 1.6.0
    Uninstalling cloudpickle-1.6.0:
      Successfully uninstalled cloudpickle-1.6.0


  Using cached https://files.pythonhosted.org/packages/e7/e3/898487e5dbeb612054cf2e0c188463acb358167fef749c53c8bb8918cea1/cloudpickle-1.6.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d6/84/759f5dd23fec8ba71952d97bcc7e2c9d7d63bdc582421f3cd4be845f0c98/gast-0.3.3-py2.py3-none-any.whl
ERROR: tensorflow-probability 0.8.0 has requirement cloudpickle==1.1.1, but you'll have cloudpickle 1.6.0 which is incompatible.
ERROR: tensorflow-probability 0.8.0 has requirement gast<0.3,>=0.2, but you'll have gast 0.3.3 which is incompatible.
  Found existing installation: cloudpickle 1.1.1
    Uninstalling cloudpickle-1.1.1:
      Successfully uninstalled cloudpickle-1.1.1
  Found existing installation: gast 0.2.2
    Uninstalling gast-0.2.2:
      Successfully uninstalled gast-0.2.2


trax                          1.3.8                


In [ ]:
import json
import random
import numpy as np
from termcolor import colored
 
import trax
from trax import layers as tl
from trax.supervised import training

!pip list | grep trax


trax                          1.3.8                


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
DATA_FILE = 'data.json'
DATA_DIR = '/content/drive/My Drive/ChatbotWork/data'

DIALOGUE_DB = {}

VOCAB_FILE = 'en_32k.subword'
VOCAB_DIR = '/content/drive/My Drive/ChatbotWork/data/vocabs'

In [ ]:
def load_json(directory, file):
    with open(f'{directory}/{file}') as file: 
        db = json.load(file)
    return db

DIALOGUE_DB = load_json(DATA_DIR, DATA_FILE)

In [ ]:
print(f'The number of dialogues is: {len(DIALOGUE_DB)}')

The number of dialogues is: 10438


In [ ]:
def get_conversation(file, data_db):
    result = ''
    
    len_msg_log = len(data_db[file]['log'])
    
    delimiter_1 = ' Person 1: '
    delimiter_2 = ' Person 2: '
    
    for i in range(len_msg_log):
        cur_log = data_db[file]['log'][i]
        
        if i%2 == 0:                   
            result += delimiter_1
        else: 
            result += delimiter_2
        
        result += cur_log['text']
    
    return result



In [ ]:
file = 'SNG01856.json'
conversation = get_conversation(file, DIALOGUE_DB)

print(conversation)

 Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel Person 2: Okay, do you have a specific area you want to stay in? Person 1: no, i just need to make sure it's cheap. oh, and i need parking Person 2: I found 1 cheap hotel for you that includes parking. Do you like me to book it? Person 1: Yes, please. 6 people 3 nights starting on tuesday. Person 2: I am sorry but I wasn't able to book that for you for Tuesday. Is there another day you would like to stay or perhaps a shorter stay? Person 1: how about only 2 nights. Person 2: Booking was successful.
Reference number is : 7GAWK763. Anything else I can do for you? Person 1: No, that will be all. Good bye. Person 2: Thank you for using our services.


In [ ]:
def print_conversation(conversation):
    
    delimiter_1 = 'Person 1: '
    delimiter_2 = 'Person 2: '
    
    split_list_d1 = conversation.split(delimiter_1)
    
    for sublist in split_list_d1[1:]:
        split_list_d2 = sublist.split(delimiter_2)
        print(colored(f'Person 1: {split_list_d2[0]}', 'red'))
        
        if len(split_list_d2) > 1:
            print(colored(f'Person 2: {split_list_d2[1]}', 'green'))

            
print_conversation(conversation)

Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel 
Person 2: Okay, do you have a specific area you want to stay in? 
Person 1: no, i just need to make sure it's cheap. oh, and i need parking 
Person 2: I found 1 cheap hotel for you that includes parking. Do you like me to book it? 
Person 1: Yes, please. 6 people 3 nights starting on tuesday. 
Person 2: I am sorry but I wasn't able to book that for you for Tuesday. Is there another day you would like to stay or perhaps a shorter stay? 
Person 1: how about only 2 nights. 
Person 2: Booking was successful.
Reference number is : 7GAWK763. Anything else I can do for you? 
Person 1: No, that will be all. Good bye. 
Person 2: Thank you for using our services.


In [ ]:
all_files = DIALOGUE_DB.keys()

untokenized_data = []

for file in all_files:
    result = get_conversation(file, DIALOGUE_DB)
    
    untokenized_data.append(result)

print(untokenized_data[0])

 Person 1: am looking for a place to to stay that has cheap price range it should be in a type of hotel Person 2: Okay, do you have a specific area you want to stay in? Person 1: no, i just need to make sure it's cheap. oh, and i need parking Person 2: I found 1 cheap hotel for you that includes parking. Do you like me to book it? Person 1: Yes, please. 6 people 3 nights starting on tuesday. Person 2: I am sorry but I wasn't able to book that for you for Tuesday. Is there another day you would like to stay or perhaps a shorter stay? Person 1: how about only 2 nights. Person 2: Booking was successful.
Reference number is : 7GAWK763. Anything else I can do for you? Person 1: No, that will be all. Good bye. Person 2: Thank you for using our services.


Split to train and eval data

In [ ]:
random.shuffle(untokenized_data)

# cutoff (5% of the total length
cut_off = int(len(untokenized_data) * .05)

train_data, eval_data = untokenized_data[:-cut_off], untokenized_data[-cut_off:]

print(f'number of conversations in the data set: {len(untokenized_data)}')
print(f'number of conversations in train set: {len(train_data)}')
print(f'number of conversations in eval set: {len(eval_data)}')

number of conversations in the data set: 10438
number of conversations in train set: 9917
number of conversations in eval set: 521


In [ ]:
def stream(data):
    while True:
        d = random.choice(data)
        
        yield (d, d)

In [ ]:
data_pipeline = trax.data.Serial(
    trax.data.Shuffle(),
    
    trax.data.Tokenize(vocab_dir=VOCAB_DIR,
                       vocab_file=VOCAB_FILE),
    
    trax.data.FilterByLength(2048),
    
    trax.data.BucketByLength(boundaries=[128, 256,  512, 1024],
                             batch_sizes=[16,    8,    4,   2, 1]),
    
    trax.data.AddLossWeights(id_to_mask=0)
)

train_stream = data_pipeline(stream(train_data))
eval_stream = data_pipeline(stream(eval_data))

In [ ]:
inp, _, _ = next(train_stream)

print("input shape: ", inp.shape)

print(trax.data.detokenize(inp[0], vocab_dir=VOCAB_DIR, vocab_file=VOCAB_FILE))

input shape:  (4, 512)
 Person 1: Hi, can you fill me in on some details about a place called jesus green outdoor pool, please? Person 2: It's a swimming pool in the North. Phone # is 01223302579 and it's between victoria road and the river. Anything else I can help you with? Person 1: That is the full address that you mentioned? Person 2: Sure.  The address showing is between victoria road and the river.  Again, their phone number is 0122330257.  Can I help with anything else? Person 1: Yes I'd like some info on a place to eat in the centre. Person 2: There are many to choose from! Did you have a type of food or price range in mind? I like Shanghai Family Restaurant, myself. It's a moderately-priced Chinese restaurant. Person 1: I'd like an expensive world food restaurant. Person 2: I am not showing an expensive world restaurant in the centre.  Would you like to try other areas? Person 1: How about one that serves gastropub food instead? Person 2: I have two places in the Centre fitti

In [ ]:
def reversible_layer_forward(x, f, g):
    x1, x2 = np.split(x, 2, axis=-1) 
        
    y1 = x1 + f(x2)
    y2 = x2 + g(y1)
    
    y = np.concatenate([y1, y2], axis=-1)
    
    return y

In [ ]:
def reversible_layer_reverse(y, f, g):
    
    y1, y2 = np.split(y, 2, axis=-1)
        
    x2 = y2 - g(y1)
    x1 = y1 - f(x2)
    
    x = np.concatenate([x1, x2], axis=-1)
    
    return x



In [ ]:
f = lambda x: x + np.random.uniform(size=x.shape)
g = lambda x: x * 3

input_vector = np.random.uniform(size=(32,))
output_vector = reversible_layer_forward(input_vector, f, g)
reversed_vector = reversible_layer_reverse(output_vector, f, g)

assert not np.allclose(reversed_vector, input_vector) 


random_seed = 27686
rng = trax.fastmath.random.get_prng(random_seed)
f = lambda x: x + trax.fastmath.random.uniform(key=rng, shape=x.shape)

output_vector = reversible_layer_forward(input_vector, f, g)
reversed_vector = reversible_layer_reverse(output_vector, f, g)

assert np.allclose(reversed_vector, input_vector,  atol=1e-07) 

In [ ]:
def ReformerLM(vocab_size=33000, n_layers=2, mode='train', attention_type=tl.SelfAttention):
    
    model = trax.models.reformer.ReformerLM( 
        vocab_size=vocab_size,
        n_layers=n_layers,
        mode=mode,
        attention_type=attention_type
    )
    
    return model

In [ ]:
temp_model = ReformerLM('train')
print(str(temp_model))

del temp_model 

Serial[
  Serial[
    ShiftRight(1)
  ]
  Embedding_train_512
  Dropout
  Serial[
    PositionalEncoding
  ]
  Dup_out2
  ReversibleSerial_in2_out2[
    ReversibleHalfResidualDecoderAttn_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidualDecoderFF_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        Serial[
          FastGelu
        ]
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidualDecoderAttn_in2_out2[
      Serial[
        LayerNorm
      ]
      SelfAttention
    ]
    ReversibleSwap_in2_out2
    ReversibleHalfResidualDecoderFF_in2_out2[
      Serial[
        LayerNorm
        Dense_2048
        Dropout
        Serial[
          FastGelu
        ]
        Dense_512
        Dropout
      ]
    ]
    ReversibleSwap_in2_out2
  ]
  Concatenate_in2
  LayerNorm
  Dropout
  Serial[
    Dense_train
  ]
]


In [ ]:
def training_loop(ReformerLM, train_gen, eval_gen, output_dir = "./model/"):

    lr_schedule = trax.lr.warmup_and_rsqrt_decay(
        n_warmup_steps=1000, max_value=0.01)
    
    
    train_task = training.TrainTask(            
        
        labeled_data=train_gen,
        
        loss_layer=tl.CrossEntropyLoss(),
        
        optimizer=trax.optimizers.Adam(0.01),
        
        lr_schedule=lr_schedule,
        
        n_steps_per_checkpoint=10
    )

    
    eval_task = training.EvalTask(                      
        
        labeled_data=eval_gen,
        
        metrics=[tl.CrossEntropyLoss(), tl.Accuracy()]
    )

    loop = training.Loop(ReformerLM(mode='train'),
                         train_task,
                         eval_tasks=[eval_task],
                         output_dir=output_dir)

    return loop

In [ ]:
def attention(*args, **kwargs):
    kwargs['predict_mem_len'] = 120
    kwargs['predict_drop_len'] = 120
    return tl.SelfAttention(*args, **kwargs)

model = ReformerLM(
    vocab_size=33000,
    n_layers=6,
    mode='predict',
    attention_type=attention,
)

shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)

In [ ]:
model.init_from_file('/content/drive/My Drive/ChatbotWork/chatbot_model1.pkl.gz', weights_only=True, input_signature=shape11)

STARTING_STATE = model.state

In [ ]:
def tokenize(sentence, vocab_file, vocab_dir):
    return list(trax.data.tokenize(iter([sentence]), vocab_file=vocab_file, vocab_dir=vocab_dir))[0]

def detokenize(tokens, vocab_file, vocab_dir):
    return trax.data.detokenize(tokens, vocab_file=vocab_file, vocab_dir=vocab_dir)

In [ ]:
def ReformerLM_output_gen(ReformerLM, start_sentence, vocab_file, vocab_dir, temperature):
    
    input_tokens = tokenize(start_sentence, vocab_file=vocab_file, vocab_dir=vocab_dir)
    
    input_tokens_with_batch = np.array(input_tokens)[None, :]
    
    output_gen = trax.supervised.decoding.autoregressive_sample_stream( 
        ReformerLM,
        inputs=input_tokens_with_batch,
        temperature=temperature
    )
        
    return output_gen

In [ ]:
import pickle

WEIGHTS_FROM_FILE = ()

with open('/content/drive/My Drive/ChatbotWork/weights', 'rb') as file:
    WEIGHTS_FROM_FILE = pickle.load(file)

shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)

def attention(*args, **kwargs):
    kwargs['predict_mem_len'] = 120
    kwargs['predict_drop_len'] = 120
    return tl.SelfAttention(*args, **kwargs)

test_model = ReformerLM(vocab_size=5, n_layers=1, mode='predict', attention_type=attention)

test_output_gen = ReformerLM_output_gen(test_model, "test", vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, temperature=0)

test_model.init_weights_and_state(shape11)

test_model.weights = WEIGHTS_FROM_FILE

output = []

for i in range(6):
    output.append(next(test_output_gen)[0])

print(output)

del test_model 
del WEIGHTS_FROM_FILE
del test_output_gen

ValueError: ignored

In [ ]:
shape11 = trax.shapes.ShapeDtype((1, 1), dtype=np.int32)

def attention(*args, **kwargs):
    kwargs['predict_mem_len'] = 120 
    kwargs['predict_drop_len'] = 120 
    return tl.SelfAttention(*args, **kwargs)

model = ReformerLM(
    vocab_size=33000,
    n_layers=6,
    mode='predict',
    attention_type=attention,
)

In [ ]:
model.init_from_file('/content/drive/My Drive/ChatbotWork/chatbot_model1.pkl.gz',
                     weights_only=True, input_signature=shape11)

STARTING_STATE = model.state

In [ ]:
def generate_dialogue(ReformerLM, model_state, start_sentence, vocab_file, vocab_dir, max_len, temperature):
    
    delimiter_1 = 'Person 1: ' 
    delimiter_2 = 'Person 2: '
    
    sentence = ''
    
    counter = 0
    
    result = [tokenize(': ', vocab_file=vocab_file, vocab_dir=vocab_dir)]
    
    ReformerLM.state = model_state
    
    output = ReformerLM_output_gen(ReformerLM, start_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, temperature=temperature)
    
    print(start_sentence.split(delimiter_2)[0].strip())
    
    for o in output:
        
        result.append(o)
        
        sentence = detokenize(np.concatenate(result, axis=0), vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR)
        
        if sentence.endswith(delimiter_1):
            sentence = sentence.split(delimiter_1)[0]
            print(f'{delimiter_2}{sentence}')
            sentence = ''
            result.clear()
        
        elif sentence.endswith(delimiter_2):
            sentence = sentence.split(delimiter_2)[0]
            print(f'{delimiter_1}{sentence}')
            sentence = ''
            result.clear()

        counter += 1
        
        if counter > max_len:
            break    



In [ ]:
sample_sentence = ' Person 1: Are there theatres in town? Person 2: '
generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=120, temperature=0.2)

Person 1: Are there theatres in town?
Person 2: : There are 4 theatres in town. Do you have a preference on area? 
Person 1: No, but I would like the one in the south. 
Person 2: I would recommend the Junction. Would you like more information on it? 
Person 1: Yes, I would like the phone number and postcode please. 
Person 1: The post code is cb17gx. The phone number is 01223511511. Can I help you with anything else? Person 43, Fensounds good! 


In [ ]:
sample_sentence = ' Person 1: Is there a college nearby? Person 2: '
generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=120, temperature=0.2)

Person 1: Is there a college nearby?
Person 2: : There are 18 colleges in Cambridge. Is there a certain area you are looking for? 
Person 1: No, I don't care. Can you recommend one and give me the address, postcode, and entrance fee for one of them? 
Person 2: I'm sorry, but I don't have any information on the entrance fees. There is no entrance fee. 


In [ ]:
sample_sentence = ' Person 1: Can you book a taxi? Person 2: '
generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=120, temperature=0.2)

Person 1: Can you book a taxi?
Person 2: : I sure can. Where are you going? 
Person 1: I'm going to be from the city stop restaurant. I need to be picked up at the scott polar museum. 
Person 2: I have a grey honda for you. The contact number is 078226756. Can I help with anything else? 
Person 1: No, that's all I need. Thank you! 
Person 1: You're welcome. Enjoy your time in Cambridge!Skunfounfortustill still still open then then then then. 


In [ ]:
sample_sentence = ' Person 1: '
sample_sentence += input()
sample_sentence += ' Person 2: '


generate_dialogue(ReformerLM=model, model_state=STARTING_STATE, start_sentence=sample_sentence, vocab_file=VOCAB_FILE, vocab_dir=VOCAB_DIR, max_len=120, temperature=0.2)

Is there a restaurant nearby?
Person 1: Is there a restaurant nearby?
Person 2: : There are 110 restaurants in Cambridge. Do you have a price range in mind? 
Person 1: I'm looking for something expensive. 
Person 2: There are no restaurants in the area. Would you like to try another type of food? 
Person 1: How about Chinese food? 
Person 2: There are 14 Chinese restaurants in town. Do you have a particular area you'd like to dine in? 
